In [1]:
from pyspark.sql import SparkSession 

sc = SparkSession.builder.appName("DataFrame").getOrCreate()#.setSystemProperty("spark.executor.memory", "4g")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/13 12:45:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [30]:
logs_interesse = [
    "Eleitor foi habilitado",
    "Voto confirmado para [Deputado Federal]",
    "Voto confirmado para [Deputado Estadual]",
    "Voto confirmado para [Senador]",
    "Voto confirmado para [Governador]",
    "Voto confirmado para [Presidente]",
    "Tecla indevida pressionada",
]

In [45]:
from pyspark.sql.functions import col
from pyspark.sql.functions import to_date
import pyspark.sql.functions as F

DATA_PATH = "/app/data/"
input_file = DATA_PATH+"texto/*" 


df = sc.read.csv(input_file, sep="\t", header=False)
df = df.drop("_c1","_c3","_c5")
df =  df.filter(col("_c4").isin(logs_interesse))
df = df.withColumn("_c0", F.to_timestamp(col("_c0"), 'dd/MM/yyyy HH:mm:ss'))

df.show(10,False)

+-------------------+--------+----------------------------------------+
|_c0                |_c2     |_c4                                     |
+-------------------+--------+----------------------------------------+
|2022-10-02 08:03:26|67305985|Eleitor foi habilitado                  |
|2022-10-02 08:03:48|67305985|Voto confirmado para [Deputado Federal] |
|2022-10-02 08:03:58|67305985|Voto confirmado para [Deputado Estadual]|
|2022-10-02 08:04:37|67305985|Voto confirmado para [Senador]          |
|2022-10-02 08:04:42|67305985|Voto confirmado para [Governador]       |
|2022-10-02 08:04:47|67305985|Voto confirmado para [Presidente]       |
|2022-10-02 08:06:26|67305985|Eleitor foi habilitado                  |
|2022-10-02 08:06:58|67305985|Voto confirmado para [Deputado Federal] |
|2022-10-02 08:07:19|67305985|Voto confirmado para [Deputado Estadual]|
|2022-10-02 08:07:25|67305985|Tecla indevida pressionada              |
+-------------------+--------+----------------------------------

In [37]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c4: string (nullable = true)



In [24]:
df.count()

1119794

In [20]:
%%time
#unique_values = df.select("_c4").distinct().orderBy("_c4")
from pyspark.sql.functions import regexp_replace
unique_values =  df.select("_c4").withColumn("_c4", regexp_replace(df["_c4"], r"\d+", "")).groupBy("_c4").count().sort("count", ascending=False)
#write as csv
unique_values.write.mode("overwrite").csv(DATA_PATH+"unique_values")
#unique_values.write.mode("overwrite").text(DATA_PATH+"unique_values")

# Show the unique values
unique_values.show()

+--------------------+------+
|                 _c4| count|
+--------------------+------+
|Tecla indevida pr...|239991|
|Solicita digital....|232247|
|Capturada a digit...|223653|
|Aguardando digita...|139922|
|Título digitado p...|135910|
|Digital capturada...|126345|
|Eleitor foi habil...|125834|
|Número de tentati...|125790|
|O voto do eleitor...|125729|
|Voto confirmado p...|125718|
|Voto confirmado p...|125683|
|Voto confirmado p...|125623|
|Voto confirmado p...|125611|
|Voto confirmado p...|125605|
|Tipo de habilitaç...| 97859|
|Batimento de digi...| 75409|
|Verificação de as...| 73218|
|Dedo reconhecido ...| 49359|
|Biometria coletad...| 40606|
|Dedo reconhecido ...| 34668|
+--------------------+------+
only showing top 20 rows

CPU times: user 28 ms, sys: 10.4 ms, total: 38.3 ms
Wall time: 6.11 s


In [46]:
sc.stop()